# Using Naive Bayes classifier and LSTM for spam detection
# Step 1 : we use the Naive Bayes Classifier

In [3]:
#data import 
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("sms_spam")
df = dataset['train'].to_pandas()



Found cached dataset sms_spam (C:/Users/shash/.cache/huggingface/datasets/sms_spam/plain_text/1.0.0/53f051d3b5f62d99d61792c91acefe4f1577ad3e4c216fb0ad39e30b9f20019c)


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
#clean and prepare data for processing

from nltk.stem import WordNetLemmatizer
a=list(df["sms"])
b=[]
for i in a :
    b.append(i.lower())

lemmatizer = WordNetLemmatizer()
def word_lemmatizer(words):
   lemma_words = [lemmatizer.lemmatize(o) for o in words]
   return "".join(lemma_words)

c = []
for i in b :
    i = word_lemmatizer(i)
    i = i.replace("1","")
    i = i.replace("2","")
    i = i.replace("3","")
    i = i.replace("4","")
    i = i.replace("5","")
    i = i.replace("6","")
    i = i.replace("7","")
    i = i.replace("8","")
    i = i.replace("9","")
    i = i.replace("0","")
    i = i.replace("!","")
    i = i.replace("@","")
    i = i.replace("#","")
    i = i.replace("$","")
    i = i.replace("%","")
    i = i.replace("^","")
    i = i.replace("&","")
    i = i.replace("*","")
    i = i.replace("(","")
    i = i.replace(")","")
    i = i.replace("_","")
    i = i.replace("-","")
    i = i.replace("+","")
    i = i.replace("=","")
    i = i.replace("?","")
    i = i.replace(".","")
    i = i.replace(",","")
    i = i.replace(":","")
    i = i.replace(";","")
    i = i.replace("\'","")
    i = i.replace("\" ","")
    i = i.replace("{","")
    i = i.replace("}","")
    i = i.replace("[","")
    i = i.replace("]","")
    i = i.replace("£","")
    i = i.replace(">","")
    i = i.replace("\x92s","")
    i = i.replace("\n","")
    i = i.replace("/","")
    c.append(i)

clean_data = c
data_label = list(df["label"])




In [6]:
#extract features from the data
from sklearn.model_selection import train_test_split as tts
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
data_vector = vectorizer.fit_transform(clean_data)

#split data into training and testing data
x_train , x_test , y_train , y_test = tts(data_vector,data_label,test_size = 0.3, random_state = 2)


In [9]:
#create naive bayes model to classify the data
import sklearn.metrics
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()
clf.fit(x_train.toarray(),y_train)
predictions = clf.predict(x_test.toarray())
print("accuracy:",clf.score(x_test.toarray(),y_test))
print("confusion matrix:",sklearn.metrics.confusion_matrix(y_test, clf.predict(x_test.toarray())))


accuracy: 0.9677226539151226
confusion matrix: [[1440    8]
 [  46  179]]


# Since the Naive Bayes classifier isn't  particularly accurate
# Especially looking at the confusion matrix
# We now use LSTM

In [10]:
#preprocessing and preparing data for out model
#tokenizer to convert the string data into numerical data
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

lb_enc = LabelEncoder()
y = lb_enc.fit_transform(df['label'])

tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_data)
text_to_sequence = tokenizer.texts_to_sequences(clean_data) 
size = len(tokenizer.word_index)+1
#padding the data so that all vectors are of the same size
max_length_sequence = max([len(i) for i in text_to_sequence])
padded_sms_sequence = pad_sequences(text_to_sequence, maxlen=max_length_sequence, padding = "pre") 


In [11]:
#creating the LSTM model
import keras
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.models import Sequential

clf = Sequential()
clf.add(Embedding(size, 32, input_length=max_length_sequence))
clf.add(LSTM(128))
clf.add(Dropout(0.4))
clf.add(Dense(units =32, activation ='relu'))
clf.add(Dropout(0.3))
clf.add(Dense(units =1 , activation = 'sigmoid'))
clf.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

clf.fit(padded_sms_sequence, y, epochs = 5, batch_size=16 ,validation_split=0.2)

Epoch 1/5
279/279 [==============================] - 49s 166ms/step - loss: 0.2038 - accuracy: 0.9374 - val_loss: 0.0689 - val_accuracy: 0.9785
Epoch 2/5
279/279 [==============================] - 34s 124ms/step - loss: 0.0392 - accuracy: 0.9883 - val_loss: 0.0535 - val_accuracy: 0.9857
Epoch 3/5
279/279 [==============================] - 35s 125ms/step - loss: 0.0182 - accuracy: 0.9962 - val_loss: 0.0644 - val_accuracy: 0.9830
Epoch 4/5
279/279 [==============================] - 33s 119ms/step - loss: 0.0088 - accuracy: 0.9980 - val_loss: 0.0642 - val_accuracy: 0.9865
Epoch 5/5
279/279 [==============================] - 32s 115ms/step - loss: 0.0182 - accuracy: 0.9946 - val_loss: 0.0768 - val_accuracy: 0.9758


In [26]:
x_train , x_test , y_train , y_test = tts(padded_sms_sequence,y,test_size = 0.3, random_state = 2)
print("accuracy",1-sklearn.metrics.log_loss(y_test,clf.predict(x_test)))

val_arr=[]
for i in clf.predict(x_test):
    if i[0] > 0.5 :
        val_arr.append(1)
    else:
        val_arr.append(0)

print("confusion matrix:",sklearn.metrics.confusion_matrix(y_test,val_arr))


53/53 [==============================] - 2s 37ms/step
accuracy 0.980353661346186
53/53 [==============================] - 3s 61ms/step
confusion matrix: [[1445    3]
 [   6  219]]


# As we can see the LSTM performs much better
# This can be further improved by tweaking the model or running more epochs